Data and Network Security HW3 Question 8

Hirbod Behnam 99171333

<div dir='rtl'>
در ابتدا ما کتابخانه‌های مورد نیاز تمرین را ایمپورت می‌کنیم.
</div>

In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import random
import math

<div dir='rtl'>
در ادامه کاری که می‌کنیم این است که دو کلید برای هر دو طرف می‌سازیم. من مثل همه‌ی داکیونتیشن‌های امنیتی از نام‌های آلیس و باب استفاده می‌کنم.
</div>

In [2]:
alice_private_key = RSA.generate(2048)
alice_public_key = alice_private_key.publickey()
bob_private_key = RSA.generate(2048)
bob_public_key = bob_private_key.publickey()

<div dir='rtl'>
در ادامه هر فرد اسم خودش را رمز می‌کند و برای آن یکی فرد می‌فرستد.
</div>

In [3]:
# Alice sends her name to Bob with his public key
alice_hello_message = PKCS1_OAEP.new(bob_public_key).encrypt(b'Alice')
print(f"Alice sent {alice_hello_message} to Bob")
bob_decrypted_message = PKCS1_OAEP.new(bob_private_key).decrypt(alice_hello_message)
print(f"Bob got '{bob_decrypted_message}' from Alice")
# Bob sends his name to Alice with her public key
bob_hello_message = PKCS1_OAEP.new(alice_public_key).encrypt(b'Bob')
print(f"Bob sent {alice_hello_message} to Alice")
alice_decrypted_message = PKCS1_OAEP.new(alice_private_key).decrypt(bob_hello_message)
print(f"Alice got '{alice_decrypted_message}' from Bob")

Alice sent b'G\x03\xc1\xd0}\xdb\x10\xb6\xca\xf7\x94\xb0\xf5\x9dh\xcf\xcb7\x8d\xbe\x0f\x90\x1a\xbf\x9f"\xc2\x8a\xb5\'\xc8y\x0e\x86\xf1\xb8\xb2\xe3K\x14\x1c\xbd\'\x96\xf7\x89\x0f\xe0\xd8&Z=]\x82\x88A\xc1\x8erWK\x8d\xfcd\xf92\x9d5(yG\xbaw\xf9\xd6\xb8\xfc\x87\x97\x17\x86\xfcTai\xb3\x84\n\xcc~\xd5\xaa\xe7\xf7T\xd4K7x\x9a\x07\x93\xb3\x16\xaa\xa2rQ@^L\'4O\x8c\xa6\x83\xcdw\xc33N\xb87\r\xef"?\x01K\x90\x0b6\xb3\xa4_[\xf5"~\x87\x96\xee\xf0\xda@"\xe1\x07nK|YI\x01\x96\xfbNp\x8f\xb8\xe6m\xf9w\xf2\x91\xbfyxDB\x90\x14\xf7\x0e\x14\ta\xc5\x13\xcf\xc3sl\xd4\xb2\xe9R\xba:@\x1e-H\xa8\x92\x8f=\x85\xcd\xb3\x94\xe4Em|\x060\xe8\xca\xd6\x85\x82\x9d3\x88\xf6A\x1cS\xe3\x12h\x05*\xe3sU\x89oX\xcc\x8f{\xa7\xd3\xd7\xa8\xc0\x87\xb6\x06=N\xea\xe6/8\x8f\xbc^\xa4\xc5\xd6)' to Bob
Bob got 'b'Alice'' from Alice
Bob sent b'G\x03\xc1\xd0}\xdb\x10\xb6\xca\xf7\x94\xb0\xf5\x9dh\xcf\xcb7\x8d\xbe\x0f\x90\x1a\xbf\x9f"\xc2\x8a\xb5\'\xc8y\x0e\x86\xf1\xb8\xb2\xe3K\x14\x1c\xbd\'\x96\xf7\x89\x0f\xe0\xd8&Z=]\x82\x88A\xc1\x8erWK\x8d\xfcd

<div dir='rtl'>
حال همان طور که گفته شده در تمرین، آلیس یک عدد تصادفی می‌سازد و آن‌ را برای باب می‌فرستد. اما نکته‌ای که هست این است در قسمت به توان رساندن بدون رمزگشایی است. عملا توابع مربوط به
PKCS1_OAEP
یک نوع
padding
درست می‌کنند که بایت‌های دیتا را تبدیل به عدد کنند. اما در این قسمت ما واقعا با خود اعداد کار داریم.
از این جهت دو تابع با توجه به ویکیپیدا نوشتم که رمزنگاری و رمزگشایی
RSA
را بر روی یک عدد انجام دهند که در زیر مشاهده می‌کنید.
</div>

In [4]:
# Encrypt just a number
def rsa_encrypt_number(data: int, key: RSA.RsaKey) -> int:
    return pow(data, key.e, key.n)

# Decrypt just a number
def rsa_decrypt_number(data: int, key: RSA.RsaKey) -> int:
    return pow(data, key.d, key.n)

<div dir='rtl'>
نکته‌ای که در اینجا مهم است این است که اگر به ویکپیدا دقت کنید متوجه می‌شوید که رمزنگاری برابر است با به توان رساندن پیام به توان
e
و رمزگشایی به توان رساندن آن عدد به
d
است. حال خاصیت ریاضی زیر را در نظر بگیرید:
</div>

$$
x^{de} = x \mod n \\
((a^e)^d)^b = ((a^b)^e)^d = a^b \mod n
$$

<div dir='rtl'>
همان طور که مشاهده می‌شود در صورتی که
a
را به توان
e
برسانیم (رمزگذاری کنیم) و سپس آن را به توان
b
برسانیم و در نهایت آن را به توان
d
برسانیم عملا همان
a به توان b
بدست می‌آید که در نهایت با لگاریتم می‌توان
b
را بدست آورد.
</div>

In [5]:
alice_random_number = random.randint(10, 100)
alice_encrypted_number = rsa_encrypt_number(alice_random_number, alice_public_key)
print(f"Alice sent {alice_encrypted_number} to Bob")
bob_power = random.randint(10, 100)
bob_powered_number = pow(alice_encrypted_number, bob_power, alice_public_key.n)
print(f"Bob sent back '{bob_powered_number}' (his power was {bob_power})")

Alice sent 6067682428421557346287813692880393964125774570580837627084088765240316832476650319361970552399539466729835593015513044753107446496439328945955731148644735773307298678982693918658540671746417662563441675292800482052156466971355908988741481635474403107973096808693572960586039338951882521025017357729476909156536480630093291642768539661043618124548462823492511396378782944729580813046992558625292544838420731533495483411686591942911487401988888305379972211329958399376637122956910056030530347013986576393799985727939234821551149747490118162570101418108533355965849940597934179427378817295660203384166337114333864462987 to Bob
Bob sent back '46197789013678926787535570992604449313019848691723875860073045093330884387710618978218473913791844933222385543918264133156574507764786172178724316313938653015442806278029878860442969823483884397785854178722608891790631281268985249121692010537109460263996133224153620338146510406000852291834731724440715996956324522259678271197164194138856972769249469

In [6]:
bob_power = rsa_decrypt_number(bob_powered_number, alice_private_key)
bob_power = int(math.log(bob_power, alice_random_number))
print(f"Bob power was {bob_power}")

Bob power was 87


<div dir='rtl'>
حال کاری که می‌کنیم این است که توان باب را برای او می‌فرستیم
(البته به صورت رمزشده با کلید خود باب)
</div>

In [7]:
alice_confirm_message = PKCS1_OAEP.new(bob_public_key).encrypt(str(bob_power).encode('utf-8'))
print(f"Alice sent {alice_confirm_message} to Bob")
bob_decrypted_message = PKCS1_OAEP.new(bob_private_key).decrypt(alice_confirm_message)
print(f"Alice sent {bob_decrypted_message} to Bob (after decryption)")

Alice sent b'6A\xee"\x1d\xae\xea\xeb\xd0\x90\xf7\xb4*\xe6\x9d\xfdA\xf9\x97\xeb\x13V\xf3\x8a\xcf\x80\x88tx~\xdc\x0c\xd8\'\x0f\xc23\xd2\xe0:\xe8\xf1H\x0e\x88\x9816k\x81p\xba\xdc=\x91\xcc\x18\xce\xb6d\xc9\x947\x05\xf1D\x9e\x92\xd0\xf2\x00\x97\x1c5\xf9\xee*\xe0\x98DT\xcfI\xe3C\x96\x8f\xd2\xaf6\xe2\x7f\xf5MJ\x1b\xfa\xf0\x1b\x94\xe39\xd4\xce\xea>\xde\xa8\x8a\x14\xcb&\x8d8\x16\x12\xa0\xe4\x88\xf2Z\xfc\x90\x10&\xe6\x9eG[\xf65\x9d\xd6s#\x89U\x06\xba\xdd\x83\xb8\x1a\x8a\xa9}U~\x91+\xe9\x17i@\x87O\xd7\x1d\xda\xc7\x0f\xa9\xd6\xa2\x0e~Y\xf6\xe2\xb8\xca\xdf\x876D\x15\xcf\x94T\x84\xb5a\x8e\x16NO\x04G#7C \xd6\x94\xae\xcb(\xc5)$%@V\x05Bn\x07\xc0\xec\x84x%\xdd\x9eD\xc5!\x05>\x03\x87\xc1\xd7\\\x0b\x1b\xf0\xf6}\xd4\x7f\xeb\xfa\xcb3R8zB\xf1\xb5\xc3\xb0F\x98\xdb\x1a\xd8\x00\xa0\xc5c\x11\xb9\x06\xa5' to Bob
Alice sent b'87' to Bob (after decryption)


<div dir='rtl'>
همان طور که مشاهده می‌کنید هر دو فرد به عدد یکسانی رسیدند. در ادامه باب یک کلید می‌سازد و برای آلیس می‌فرستد.
</div>

In [8]:
session_key_bob = get_random_bytes(32)
iv_bob = get_random_bytes(16)
session_key_encrypted = PKCS1_OAEP.new(alice_public_key).encrypt(session_key_bob + iv_bob)
print(f"Encoded session key is {session_key_encrypted}")
session_key_alice = PKCS1_OAEP.new(alice_private_key).decrypt(session_key_encrypted)
iv_alice = session_key_alice[-16:]
session_key_alice = session_key_alice[:32]
print(f"Alice has {session_key_alice} and Bob has {session_key_bob}")

Encoded session key is b'\x8f\xb7.\x7f,)\xd3!\xd3\x02\x01\xc4\x1f\xa6\xa4\xf4\xc3\xa3\xec\x0eq\x19\xe6\xe3\xc6\x8c\xcdt\x9f\x80\xa8\xddJ,\xfd.\xf0k\xb1\xae\xc3W\xdfkc<[\x97\xa7C\xb5q1I\xfe\xc6\x9b\x013\xf9\xbd\x11e8&W\xdc\x92\xb3\xed\xaf\xaf\x1c4i5\xea\x9e\xb9_\'i\x15\x87\xa4\xe6\x89\xad,O\xd9t]\xfc\xa4\xa7L4\xae7\xab\n+%\x81g9\x17u\x01\x0es\xa4\xa9\x94\xed\x0e\xc6v\xe3;\xb2\x81\x05U\xfeI\x11\xe8\xc2\xaa\'\xc9\xf4\x99\x9fC\xda\x91\xc4\x90\xb6\xf5-\xf8\xe4\xcc\xc5\x1c\xd5i\xa6-\x9e^@V\xd6\x97f=\xfe|\xff\xa0\x0f\x90J\x03\xe4\xa7\xce?J\x82\xec\xa9\xc6T\x9f(\xb67\xddWiz\xc8t\n\xa5\xfc\xe93e4\xbft\x9b\xc0<C\x15om\xd92%Q\xb0\xf2\xdeyG(\x84\x02.F\x04\xdd\x14\xfb~\xab$P\xb8\xf8F\x01\x0bCrN\xf1\x90U\xb7\xd5q&\x19$O"\xd9\xd9\x02;%+\xa0\xfedq'
Alice has b'\xefn\xff(5\xc8\x12S\xfd\xf8\x9b\xa3\x99\xd9E\xafn\xaaR5\x86\xf2X\r\x93-\xed\x8d\xe9\xd4\x95\x8c' and Bob has b'\xefn\xff(5\xc8\x12S\xfd\xf8\x9b\xa3\x99\xd9E\xafn\xaaR5\x86\xf2X\r\x93-\xed\x8d\xe9\xd4\x95\x8c'


<div dir='rtl'>
حال هر دو نفر یک کلید یکسان دارند و با
AES
پیامی را رمز می‌کنیم.
</div>

In [17]:
alice_cipher = AES.new(session_key_alice, AES.MODE_CBC, iv_alice)
bob_cipher = AES.new(session_key_bob, AES.MODE_CBC, iv_bob)
alice_encrypted = alice_cipher.encrypt(pad(b"Salaam!", 16))
print(f"Alice has sent {alice_encrypted}")
print(f"Bob decrypted it as {unpad(bob_cipher.decrypt(alice_encrypted), 16)}")

Alice has sent b'\x84Z\xd33\xbb\xae\xc9#6\xaam\xa4\x12a\xb1D'
Bob decrypted it as b'Salaam!'
